# Quick Start Guide (Notebook)

Minimal, runnable version of the quick-start example. Toggle sampler flags to run heavier demos; defaults keep execution light. Standard priors are pulled from Discovery's ``priordict_standard`` via ``standard_priors``.


In [1]:
# Environment and toggles
RUN_NESSAI = True
RUN_JAXNS = True
RUN_ERYN = True
RUN_GPRY = True


In [2]:
import json
import re
import numpy as np
import jax
import discovery as ds
from discoverysamplers.priors import standard_priors
import os
import glob

jax.config.update("jax_enable_x64", True)

# Path to a bundled pulsar file (relative to the discovery package install)
path_to_discovery = ds.__path__[0]
FEATHER_PATH = os.path.join(path_to_discovery, "../../data")
print(f"Using pulsar path: {FEATHER_PATH}")


Using pulsar path: /Users/jeg/Documents/discovery/discovery/src/discovery/../../data


In [3]:
# Load pulsar and build a likelihood
allpsrs = [ds.Pulsar.read_feather(psrfile) for psrfile in sorted(glob.glob(os.path.join(FEATHER_PATH,'*-[JB]*.feather')))]
psr = allpsrs[0]
likelihood = ds.PulsarLikelihood([
    psr.residuals,
    ds.makenoise_measurement_simple(psr),
])
params = likelihood.logL.params
print(f"Discovered {len(params)} parameters")
print("Parameters:", params)


Discovered 2 parameters
Parameters: ['B1855+09_efac', 'B1855+09_log10_t2equad']


In [4]:
# Standard priors from discovery (uniform bounds defined in priordict_standard)
priors = standard_priors(params)

print("Standard priors:")
for p in params:
    print(f"  {p}: {priors[p]}")

Standard priors:
  B1855+09_efac: {'dist': 'uniform', 'min': 0.9, 'max': 1.1}
  B1855+09_log10_t2equad: {'dist': 'uniform', 'min': -8.5, 'max': -5.0}


In [5]:
true_vals = {
    "B1855+09_log10_t2equad": -6.517929916655293
}

In [6]:
# Get the true values for the parameters and fix all but the first two

fixed_params = params[1:]
for name in fixed_params:
    if name in priors:
        priors[name] = {"dist": "fixed", "value": true_vals[name]}

print("Prior with fixed parameters")
for p in params:
    print(f"  {p}: {priors[p]}")

Prior with fixed parameters
  B1855+09_efac: {'dist': 'uniform', 'min': 0.9, 'max': 1.1}
  B1855+09_log10_t2equad: {'dist': 'fixed', 'value': -6.517929916655293}


In [7]:
print(true_vals)

{'B1855+09_log10_t2equad': -6.517929916655293}


In [8]:
# Quick sanity check: draw one sample from the priors and evaluate the likelihood
rng = np.random.default_rng(123)
sample = {}
for name, spec in priors.items():
    dist = spec["dist"]
    if dist == "uniform":
        sample[name] = rng.uniform(spec["min"], spec["max"])
    elif dist == "loguniform":
        sample[name] = np.exp(rng.uniform(np.log(spec["min"]), np.log(spec["max"])))
    elif dist == "fixed":
        sample[name] = spec["value"]
    else:
        raise ValueError(f"Unsupported dist in this quick demo: {dist}")
print("Log-likelihood for one random draw:", likelihood.logL(sample))

# Test edge cases
print("\nTesting edge cases:")
for efac_val in [0.9, 0.95, 1.0, 1.05, 1.1]:
    test_sample = {'B1855+09_efac': efac_val, 'B1855+09_log10_t2equad': true_vals['B1855+09_log10_t2equad']}
    logL = likelihood.logL(test_sample)
    print(f"  efac={efac_val}: logL={logL}")


Log-likelihood for one random draw: 97446.43448870355

Testing edge cases:
  efac=0.9: logL=96186.9572927178
  efac=0.95: logL=96748.58836262813
  efac=1.0: logL=97188.28908331192
  efac=1.05: logL=97530.62052096789
  efac=1.1: logL=97794.5188412677


## Nessai (optional)

In [9]:

if RUN_NESSAI:
    try:
        from discoverysamplers.nessai_interface import DiscoveryNessaiBridge
        bridge = DiscoveryNessaiBridge(discovery_model=likelihood, priors=priors, jit=True)
        results = bridge.run_sampler(
            nlive=100,
            output="output/nessai_demo", 
            resume=False
        )
        print("Nessai logZ:", results["logZ"], "+/-", results.get("logZ_err"))
    except Exception as exc:
        print("Nessai run skipped due to error:", exc)
else:
    print("RUN_NESSAI=False; skipping Nessai demo")


Nessai run skipped due to error: names list has length 1. nessai is not designed to handle one-dimensional models due to limitations imposed by the normalising flow-based proposals it uses. Consider using other methods instead of nessai.


## JAX-NS (optional)

In [10]:

if RUN_JAXNS:
    try:
        # Force reload the module to pick up changes
        import importlib
        import discoverysamplers.jaxns_interface
        importlib.reload(discoverysamplers.jaxns_interface)
        from discoverysamplers.jaxns_interface import DiscoveryJAXNSBridge
        
        # Create bridge
        bridge = DiscoveryJAXNSBridge(discovery_model=likelihood, priors=priors, latex_labels=None, jit=True)
        
        print("Testing with max_samples=5000:")
        results = bridge.run_sampler(nlive=200, max_samples=5000, termination_frac=0.05, rng_seed=0)
        
        print(f"JAX-NS logZ: {results['logZ']:.2f} +/- {results['logZerr']:.3f}")
        print(f"Actually used {results['state'].num_samples} out of 5000 allocated samples")
        
    except Exception as exc:
        import traceback
        print("JAX-NS run skipped due to error:")
        traceback.print_exc()
else:
    print("RUN_JAXNS=False; skipping JAX-NS demo")


INFO:jaxns:Number of Markov-chains set to: 200


Testing with max_samples=5000:
JAX-NS logZ: 97796.52 +/- 0.019
Actually used 2700 out of 5000 allocated samples
JAX-NS logZ: 97796.52 +/- 0.019
Actually used 2700 out of 5000 allocated samples


## Eryn MCMC (optional)

In [11]:

if RUN_ERYN:
    try:
        from discoverysamplers.eryn_interface import DiscoveryErynBridge
        
        bridge = DiscoveryErynBridge(model=likelihood, priors=priors)
        nwalkers = max(2 * bridge.ndim, 32)
        
        # Create sampler with parallel tempering (optional)
        sampler = bridge.create_sampler(nwalkers=nwalkers, tempering_kwargs=dict(ntemps=3))
        
        # Run MCMC 
        bridge.run_sampler(nsteps=100, progress=False)
        
        # Get samples using the bridge's method
        samples = bridge.return_sampled_samples()
        print(f"Eryn chain shape: {samples['chain'].shape} (nsteps, ntemps, nwalkers, nleaves, ndim)")
        print(f"Sampled parameters: {samples['names']}")
        print(f"Fixed parameters: {bridge.fixed_param_dict}")
    except Exception as exc:
        import traceback
        print("Eryn run skipped due to error:")
        traceback.print_exc()
else:
    print("RUN_ERYN=False; skipping Eryn demo")


Eryn chain shape: (100, 3, 32, 1, 1) (nsteps, ntemps, nwalkers, nleaves, ndim)
Sampled parameters: ['B1855+09_efac']
Fixed parameters: {'B1855+09_log10_t2equad': -6.517929916655293}


## GPry via Cobaya (optional)

In [12]:

if RUN_GPRY:
    try:
        from discoverysamplers.gpry_interface import DiscoveryGPryCobayaBridge
        
        bridge = DiscoveryGPryCobayaBridge(
            discovery_model=likelihood.logL, 
            priors=priors, 
            like_name="pulsar_likelihood",
        )
        
        print(f"Original param names: {bridge.orig_param_names}")
        print(f"Sanitized param names: {bridge.sanitized_param_names}")
        print(f"Sampled (original): {bridge.sampled_names}")
        print(f"Fixed (original): {bridge.fixed_names}")
        
        info, runner = bridge.run_sampler()
        print(f"\nGPry runner completed successfully!")
    except Exception as exc:
        import traceback
        print("GPry run skipped due to error:")
        traceback.print_exc()
else:
    print("RUN_GPRY=False; skipping GPry demo")


INFO:pulsar_likelihood:Initialized external likelihood.


[pulsar_likelihood] Initialized external likelihood.
Original param names: ['B1855+09_efac', 'B1855+09_log10_t2equad']
Sanitized param names: ['B1855_09_efac']
Sampled (original): ['B1855+09_efac']
Fixed (original): ['B1855+09_log10_t2equad']
Original param names: ['B1855+09_efac', 'B1855+09_log10_t2equad']
Sanitized param names: ['B1855_09_efac']
Sampled (original): ['B1855+09_efac']
Fixed (original): ['B1855+09_log10_t2equad']
Initializing SurrogateModel with the following options:
* X-preprocessor: NormalizeBounds
* y-preprocessor: NormalizeY
* GPR kernel:
   3.16**2 * RBF(length_scale=0.1)
  with hyperparameters (in transformed scale):
    -Hyperparameter(name='k1__constant_value', value_type='numeric', bounds=array([[1.e-04, 1.e+06]]), max_length=None, n_elements=1, fixed=False, dynamic=False)
    -Hyperparameter(name='k2__length_scale', value_type='numeric', bounds=array([[1.e-05, 1.e+05]]), max_length=None, n_elements=1, fixed=False, dynamic=False)
* Noise level: 0.01
* Classifi

4it [00:00, 549.98it/s]              

[EVALUATION] (0.042 sec) Evaluated the true log-posterior at 12 location(s), of which 4 returned a finite value.
[FIT] (0.051 sec) Fitted GP model with new acquired points, including the surrogate model hyperparameters. 4 finite points were added to the GP regressor. Hyperparameters were fit with 12 restart(s).
| Iteration 1 (at most 58 left)                                               |
| Total truth evals: 12 (4 finite) of 70                                      |
1 of the proposed points are already in the training set. Skipping them.
[ACQUISITION] (0.032 sec) Proposed 0 point(s) for truth evaluation.
Acquisition returned less than half of the requested points. Re-sampling (1 tries remaining)
| Iteration 2 (at most 58 left)                                               |
| Total truth evals: 12 (4 finite) of 70                                      |
1 of the proposed points are already in the training set. Skipping them.
[ACQUISITION] (0.02 sec) Proposed 0 point(s) for truth evalu